# Webcam

In [8]:
# image processing library for python 
import cv2

In [9]:
'''the input:
1. video url / video path
2. int > the number of webcam of laptop
3. write the rtsp url to connect to camera
'''
cap = cv2.VideoCapture(0)
frameShape = None
returnCode = None

while True:
    # read one frame > ret show that the reading was done successfully or not
    ret, frame = cap.read()
    # frame is numpy array H*W*#channels
    cv2.imshow('Webcam', frame)
    # 13 is the int code for Enter
    if cv2.waitKey(1) == 13:
        returnCode = ret
        frameShape = frame.shape
        break

# Release camera and close the windows
# The webcam is an exclusive resource that must be released
cap.release()
cv2.destroyAllWindows()
print(returnCode)
print(frameShape)

True
(480, 640, 3)


# MobileNet


In [ ]:
!pip install keras
!pip install tensorflow

In [ ]:
!pip install --upgrade tensorflow

In [15]:
'''
MobileNet is a deep-learning computer vision model architecture that is 
designed for efficient and lightweight computation on mobile devices. 
It was developed by Google researchers in 2017 and is used for a variety 
of computer vision tasks such as image classification, object detection, 
and face recognition.
'''
from tensorflow.keras.applications import MobileNetV2
# from keras.preprocessing import image
# from keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
# import numpy as np

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [3]:

model = MobileNetV2(weights='imagenet')


In [4]:
def inference(x):
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return decode_predictions(preds, top=1)[0][0][1]

In [5]:
img_path = './image/elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)

In [6]:
inference(x)

'African_elephant'

In [7]:
import cv2
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (224, 224))
    predicted = inference(frame[...,::-1])
    cv2.putText(frame,  predicted, (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), lineType=cv2.LINE_AA)
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
cap.release()
cv2.destroyAllWindows()     

<div class="alert alert-block alert-info">
<div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma"> دوره مقدماتی یادگیری عمیق<br>علیرضا اخوان پور<br>  23 و 30 آبان 1397<br>
</div>
<a href="http://class.vision">Class.Vision</a> - <a href="http://AkhavanPour.ir">AkhavanPour.ir</a> - <a href="https://github.com/Alireza-Akhavan/">GitHub</a>

</div>